# Data Analyst - Challenge

Business Scenario
You are a data analyst and have a large ecommerce company in India (let’s call it X) as a client.
X gets a few thousand orders via their website on a daily basis and they have to deliver them as
fast as they can. For delivering the goods ordered by the customers, X has tied up with multiple
courier companies in India who charge them some amount per delivery.
The charges are dependent upon two factors:
● Weight of the product
● Distance between the warehouse (pickup location) and customer’s delivery address
(destination location)
On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders
per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly
basis as charges.
As the amount that X has to pay to the courier companies is very high, they want to verify if the
charges levied by their partners per Order are correct.
Input Data
Left Hand Side (LHS) Data (X’s internal data spread across three reports)
● Website order report which will list Order IDs and various products (SKUs) part of each
order. Order ID is common identifier between X’s order report and courier company
invoice
● SKU master with gross weight of each product. This should be used to calculate total
weight of each order and during analysis compare against one reported by courier
company in their CSV invoice per Order ID. The courier company calculates weight in
slabs of 0.5 KG multiples, so first you have to figure out the total weight of the shipment
and then figure out applicable weight slabs.
For example:
- If the total weight is 400 gram then weight slab should be 0.5
- If the total weight is 950 gram then weight slab should be 1
- If the total weight is 1 KG then weight slab should be 1
- If the total weight is 2.2 KG then weight slab should be 2.5

Warehouse pincode to All India pincode mapping (this should be used to figure out
delivery zone (a/b/c/d/e) and during analysis compare against one reported by courier
company in their CSV invoice per Order ID
RHS Data (courier company invoice in CSV file)
● Invoice in CSV file mentioning AWB Number (courier company’s own internal ID), Order
ID (company X’s order ID), weight of shipment, warehouse pickup pincode, customer
delivery pincode, zone of delivery, charges per shipment, type of shipment
● Courier charges rate card at weight slab and pincode level. If the invoice mentions
“Forward charges” then only forward charges (“fwd”) should be applicable as per zone
and fixed & additional weights based on weight slabs. If the invoice mentions “Forward
and rto charges” then forward charges (“fwd”) and RTO charges (“rto”) should be
applicable as per zone and fixed & additional weights based on weight slabs.
● For the first 0.5 KG, “fixed” rate as per the slab is applicable. For each additional 0.5 KG,
“additional” weight in the same proportion is applicable. Total charges will be “fixed” +
“total additional” if any

Output Data 1
Create a resultant CSV/Excel file with the following columns:
● Order ID
● AWB Number
● Total weight as per X (KG)
● Weight slab as per X (KG)
● Total weight as per Courier Company (KG)
● Weight slab charged by Courier Company (KG)
● Delivery Zone as per X
● Delivery Zone charged by Courier Company
● Expected Charge as per X (Rs.)
● Charges Billed by Courier Company (Rs.)
● Difference Between Expected Charges and Billed Charges (Rs.)

Output Data 2
Create a summary table

Count Amount (Rs.)
Total orders where X has been correctly charged <count> <total invoice
amount>

Total Orders where X has been overcharged <count> <total
overcharging
amount>
Total Orders where X has been undercharged <count> <total
undercharging
amount>

Website: cointab.in Email: work@cointab.in
Challenge Data Download
Please download the challenge data from the following link:
https://drive.google.com/file/d/1NjYY0t8ed4fcLEvgQl9DIqFluUkVL3GF/view

# SOLUTION:-

In [1]:
#import library
import pandas as pd
import numpy as np

# Company X - Order Report.csv

In [2]:
#read Company X - Report.csv file by pandas
df1=pd.read_csv('Company X - Order Report.csv')

In [3]:
#print df1 
df1

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [4]:
#Change the name of the column "ExtreamOrderNo" into "Order ID"
df1=df1.rename(columns={"ExternOrderNo":"Order ID"})

In [5]:
#checking null values in the dataset
df1.isnull().sum()

Order ID     0
SKU          0
Order Qty    0
dtype: int64

In [6]:
#no null value contains

In [7]:
#checking info of df1 
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Order ID   400 non-null    int64  
 1   SKU        400 non-null    object 
 2   Order Qty  400 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.5+ KB


In [8]:
#Checking Unique Values in the DataFrame
print(df1.nunique())

Order ID     124
SKU           65
Order Qty      6
dtype: int64


In [9]:
#checking unique value in Order ID column
df1["Order ID"].value_counts()

2001809820    14
2001821185     8
2001817093     8
2001827036     8
2001812854     8
              ..
2001820690     1
2001806226     1
2001821679     1
2001806823     1
2001806686     1
Name: Order ID, Length: 124, dtype: int64

In [10]:
#Duplicate value is present

In [11]:
#checking some duplicate values to see others data of that rows are same or not


In [12]:
df1[df1["Order ID"].duplicated()]

,Order ID,SKU,Order Qty
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
5,2001827036,GIFTBOX202002,1.0
...,...,...,...
391,2001806233,8904223819260,1.0
393,2001806232,8904223819147,2.0
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0


In [13]:
df1[df1["Order ID"]==2001809820]

,Order ID,SKU,Order Qty
158,2001809820,8904223819277,1.0
159,2001809820,8904223818478,1.0
160,2001809820,8904223819284,1.0
161,2001809820,8904223819130,1.0
162,2001809820,8904223819031,2.0
163,2001809820,8904223819024,2.0
164,2001809820,8904223816214,1.0
165,2001809820,8904223818874,1.0
166,2001809820,8904223818881,1.0
167,2001809820,8904223818898,1.0


In [14]:
df1[df1["Order ID"]==2001821185]

,Order ID,SKU,Order Qty
33,2001821185,8904223818942,2.0
34,2001821185,8904223818683,2.0
35,2001821185,8904223819239,1.0
36,2001821185,8904223819246,1.0
37,2001821185,8904223819253,1.0
38,2001821185,8904223818669,1.0
39,2001821185,8904223819147,1.0
40,2001821185,8904223818850,1.0


#we can see that in the dataframe product SKU and order qty is differnt but the order id is same. We want Order id as a primary key

# Company X - Pincode Zones.csv

In [16]:
#Read Company X - Pincode Zones.csv file by pandas 
df2=pd.read_csv("Company X - Pincode Zones.csv")

In [17]:
df2

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [18]:
#Checking unique value in each columns
df2.nunique()

Warehouse Pincode      1
Customer Pincode     108
Zone                   3
dtype: int64

In [19]:
df2['Zone'].value_counts()

b    79
d    38
e     7
Name: Zone, dtype: int64

In [20]:
#checking missing value
df2.isnull().sum()

Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64

In [21]:
#change the name of column "Zone" into "Delivery Zone as per X"
df2=df2.rename(columns={"Zone":"Delivery Zone as per X"})

In [22]:
df2

,Warehouse Pincode,Customer Pincode,Delivery Zone as per X
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [23]:
#Since 121003 pincode contains only "Warehouse Pincode", we droped this column
df2=df2.drop(["Warehouse Pincode"],axis=1)

In [24]:
df2

,Customer Pincode,Delivery Zone as per X
0,507101,d
1,486886,d
2,532484,d
3,143001,b
4,515591,d
...,...,...
119,325207,b
120,303702,b
121,313301,b
122,173212,e


In [25]:
#Checking Number of unique value in each column
df2.nunique()

Customer Pincode          108
Delivery Zone as per X      3
dtype: int64

# Company X - SKU Master.csv

In [26]:
#insert company X - SKU Master.csv File 
df3=pd.read_csv('Company X - SKU Master.csv')

In [27]:
df3

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [28]:
#Checking null values
df3.isnull().sum()

SKU           0
Weight (g)    0
dtype: int64

In [29]:
#Checking unique values in each column
df3.nunique()

SKU           65
Weight (g)    31
dtype: int64

In [30]:
#checking unique value in SKU column
df3["SKU"].value_counts()

GIFTBOX202002    2
8904223818645    1
8904223819468    1
8904223819017    1
8904223815873    1
                ..
8904223819093    1
8904223819246    1
8904223819161    1
8904223819338    1
8904223819369    1
Name: SKU, Length: 65, dtype: int64

In [32]:
df3[df3['SKU']=='GIFTBOX202002']

,SKU,Weight (g)
52,GIFTBOX202002,500
56,GIFTBOX202002,500


In [33]:
#since row no 52 and 56 are same, we droped one of them
df3.drop_duplicates(subset="SKU", inplace=True)

In [34]:
df3

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [35]:
#checking unique value in SKU column
df3["SKU"].value_counts()

8904223818645    1
GIFTBOX202001    1
8904223819017    1
8904223815873    1
8904223818478    1
                ..
8904223819093    1
8904223819246    1
8904223819161    1
8904223819338    1
8904223819369    1
Name: SKU, Length: 65, dtype: int64

In [36]:
#No duplicate value 

In [84]:
#recheck df1 file for marging with df3 file
df1

,Order ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [37]:
#Marging df1 and df3 file on "SKU" Column
df6=pd.merge(df1,df3, on= "SKU")

In [38]:
#Marging Dataframe df6
df6

,Order ID,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127
...,...,...,...,...
395,2001806616,8904223819123,1.0,250
396,2001806567,8904223815804,1.0,160
397,2001806567,8904223818577,1.0,150
398,2001806408,8904223819437,2.0,552


In [39]:
#Check for null values
df6.isnull().sum()

Order ID      0
SKU           0
Order Qty     0
Weight (g)    0
dtype: int64

In [ ]:
#No null values exist

In [40]:
#Checking Null Values
df6.nunique()

Order ID      124
SKU            65
Order Qty       6
Weight (g)     31
dtype: int64

In [41]:
#Add a new column "Total weight as per X (G)" from "Weight (g)" and "Order Qty"
df6["Total weight as per X (G)"]=df6["Weight (g)"]*df6["Order Qty"]

In [42]:
df6

,Order ID,SKU,Order Qty,Weight (g),Total weight as per X (G)
0,2001827036,8904223818706,1.0,127,127.0
1,2001821995,8904223818706,1.0,127,127.0
2,2001819252,8904223818706,1.0,127,127.0
3,2001816996,8904223818706,1.0,127,127.0
4,2001814580,8904223818706,1.0,127,127.0
...,...,...,...,...,...
395,2001806616,8904223819123,1.0,250,250.0
396,2001806567,8904223815804,1.0,160,160.0
397,2001806567,8904223818577,1.0,150,150.0
398,2001806408,8904223819437,2.0,552,1104.0


In [91]:
#changing column name
df6=df6.rename(columns={"Total weight as per X (G)" : "Total weight as per X (KG)"})

In [92]:
df6

,Order ID,SKU,Order Qty,Weight (g),Total weight as per X (KG)
0,2001827036,8904223818706,1.0,127,127.0
1,2001821995,8904223818706,1.0,127,127.0
2,2001819252,8904223818706,1.0,127,127.0
3,2001816996,8904223818706,1.0,127,127.0
4,2001814580,8904223818706,1.0,127,127.0
...,...,...,...,...,...
395,2001806616,8904223819123,1.0,250,250.0
396,2001806567,8904223815804,1.0,160,160.0
397,2001806567,8904223818577,1.0,150,150.0
398,2001806408,8904223819437,2.0,552,1104.0


In [43]:
#Checking info for changing unit g to Kg
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Order ID                   400 non-null    int64  
 1   SKU                        400 non-null    object 
 2   Order Qty                  400 non-null    float64
 3   Weight (g)                 400 non-null    int64  
 4   Total weight as per X (G)  400 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 18.8+ KB


In [44]:
#changing column name
df6=df6.rename(columns={"Total weight as per X (G)" : "Total weight as per X (KG)"})

#unit change G to KG
df6["Total weight as per X (KG)"]=np.round(df6['Total weight as per X (KG)']/1000,3)

In [45]:
df6

,Order ID,SKU,Order Qty,Weight (g),Total weight as per X (KG)
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127
2,2001819252,8904223818706,1.0,127,0.127
3,2001816996,8904223818706,1.0,127,0.127
4,2001814580,8904223818706,1.0,127,0.127
...,...,...,...,...,...
395,2001806616,8904223819123,1.0,250,0.250
396,2001806567,8904223815804,1.0,160,0.160
397,2001806567,8904223818577,1.0,150,0.150
398,2001806408,8904223819437,2.0,552,1.104


In [46]:
#we have 400 rows but Unique Order ID only 124. Now we can do the below process

In [47]:
df6=df6.groupby(['Order ID']).sum()

In [48]:
df6

,Order Qty,Weight (g),Total weight as per X (KG)
Order ID,,,
2001806210,2.0,220,0.220
2001806226,2.0,240,0.480
2001806229,3.0,500,0.500
2001806232,8.0,377,1.302
2001806233,2.0,245,0.245
...,...,...,...
2001821995,2.0,477,0.477
2001822466,10.0,352,1.376
2001823564,6.0,336,0.672


In [50]:
#Now we have only 124 unique order id and it's total weight

In [51]:
#reset index "Order ID"
df6.reset_index(level=0, inplace=True)
df6

,Order ID,Order Qty,Weight (g),Total weight as per X (KG)
0,2001806210,2.0,220,0.220
1,2001806226,2.0,240,0.480
2,2001806229,3.0,500,0.500
3,2001806232,8.0,377,1.302
4,2001806233,2.0,245,0.245
...,...,...,...,...
119,2001821995,2.0,477,0.477
120,2001822466,10.0,352,1.376
121,2001823564,6.0,336,0.672
122,2001825261,13.0,611,1.557


In [52]:
#we need only "Order ID" and Total weight(KG) for next step. So, we droped the Other Columns
df6_new=df6.loc[:,["Order ID","Total weight as per X (KG)"]]
df6_new

,Order ID,Total weight as per X (KG)
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245
...,...,...
119,2001821995,0.477
120,2001822466,1.376
121,2001823564,0.672
122,2001825261,1.557


In [54]:
#Creating weght slab as per X  #Using for loop

In [55]:
list=[]
for i in df6_new["Total weight as per X (KG)"]:
    if (i-int(i))<=0.5 and (i-int(i))>0.0:
        k=np.round(int(i)+0.5,1)
        list.append(k)
    elif (i-int(i))>0.5:
        k=np.round(int(i)+1,1)
        list.append(k)
    elif (i-int(i))==0:
        k=np.round(int(i),1)
        list.append(k)

In [56]:
#weight slab list
list

[0.5,
 0.5,
 0.5,
 1.5,
 0.5,
 0.5,
 1,
 0.5,
 0.5,
 2.5,
 0.5,
 1,
 2,
 0.5,
 0.5,
 1,
 0.5,
 1,
 0.5,
 0.5,
 0.5,
 1,
 0.5,
 1,
 1,
 0.5,
 0.5,
 0.5,
 1,
 0.5,
 0.5,
 0.5,
 1.5,
 1.5,
 0.5,
 0.5,
 1,
 0.5,
 0.5,
 0.5,
 2.5,
 1.5,
 1,
 0.5,
 1,
 0.5,
 0.5,
 1,
 1.5,
 1,
 1,
 1,
 0.5,
 1.5,
 0.5,
 1,
 1,
 0.5,
 0.5,
 1,
 1,
 0.5,
 1.5,
 0.5,
 0.5,
 1,
 1,
 0.5,
 0.5,
 0.5,
 0.5,
 1,
 2,
 2,
 3.5,
 1,
 0.5,
 1,
 0.5,
 0.5,
 1,
 1,
 0.5,
 0.5,
 1,
 1.5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.5,
 0.5,
 1,
 1,
 3,
 0.5,
 1,
 0.5,
 0.5,
 1,
 1,
 0.5,
 1.5,
 1,
 1,
 0.5,
 0.5,
 1,
 2.5,
 1.5,
 0.5,
 1,
 0.5,
 0.5,
 1,
 0.5,
 0.5,
 1.5,
 1,
 2,
 2]

In [57]:
#Checking length
len(list)

124

In [58]:
#add  a new column "Weight slab as per X (KG)"
df6_new["Weight slab as per X (KG)"]=list

In [59]:
df6_new

,Order ID,Total weight as per X (KG),Weight slab as per X (KG)
0,2001806210,0.220,0.5
1,2001806226,0.480,0.5
2,2001806229,0.500,0.5
3,2001806232,1.302,1.5
4,2001806233,0.245,0.5
...,...,...,...
119,2001821995,0.477,0.5
120,2001822466,1.376,1.5
121,2001823564,0.672,1.0
122,2001825261,1.557,2.0


# Courier Company - Invoice.csv

In [60]:
#import Courier Company - Invoice,csv File
df4=pd.read_csv("Courier Company - Invoice.csv")

In [61]:
df4

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0


In [62]:
#Checking Unique values in each Column
print(df4.nunique())

AWB Code                124
Order ID                124
Charged Weight           54
Warehouse Pincode         1
Customer Pincode        108
Zone                      3
Type of Shipment          2
Billing Amount (Rs.)     20
dtype: int64


In [63]:
#We get 124 unique Order ID from courier Company invoice 

In [64]:
#checking unique value in Type of Shipment column
df4["Type of Shipment"].value_counts()

Forward charges            109
Forward and RTO charges     15
Name: Type of Shipment, dtype: int64

In [65]:
#Recheck df6_new data for marging with df4
df6_new

,Order ID,Total weight as per X (KG),Weight slab as per X (KG)
0,2001806210,0.220,0.5
1,2001806226,0.480,0.5
2,2001806229,0.500,0.5
3,2001806232,1.302,1.5
4,2001806233,0.245,0.5
...,...,...,...
119,2001821995,0.477,0.5
120,2001822466,1.376,1.5
121,2001823564,0.672,1.0
122,2001825261,1.557,2.0


In [66]:
#marging df4 and df6_new on "Order ID" column
df_new=pd.merge(df4,df6_new, on= "Order ID")

In [67]:
df_new

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Total weight as per X (KG),Weight slab as per X (KG)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5
...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.607,1.0
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,0.515,1.0
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,0.689,1.0


In [68]:
#Rename column
df_new=df_new.rename(columns={"Charged Weight":"Total weight as per Courier Company (KG)","Zone":"Delivery Zone charged by Courier Company","Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.)"})

In [69]:
#Checking Dataframe after renaming
df_new

,AWB Code,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5
...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.607,1.0
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,0.515,1.0
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,0.689,1.0


In [70]:
#creating weight slab as per curier company 
list_new=[]
for i in df_new["Total weight as per Courier Company (KG)"]:
    if (i-int(i))<=0.5 and (i-int(i))>0.0:
        k=np.round(int(i)+0.5,1)
        list_new.append(k)
    elif (i-int(i))>0.5:
        k=np.round(int(i)+1,1)
        list_new.append(k)
    elif (i-int(i))==0:
        k=np.round(int(i),1)
        list_new.append(k)

In [71]:
list_new

[1.5,
 1,
 2.5,
 1,
 0.5,
 0.5,
 1,
 1.5,
 0.5,
 0.5,
 1,
 1,
 1.5,
 1,
 0.5,
 1.5,
 0.5,
 1,
 0.5,
 1,
 1,
 1.5,
 1,
 1,
 1,
 1,
 1,
 1.5,
 1.5,
 1,
 2,
 1.5,
 1,
 3,
 1,
 1,
 1,
 1.5,
 1,
 1,
 1,
 1,
 1.5,
 1,
 1.5,
 1,
 1.5,
 1,
 1,
 3,
 1.5,
 1,
 2,
 1,
 2,
 1,
 1.5,
 1.5,
 3,
 2,
 1,
 0.5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.5,
 3,
 1,
 1,
 0.5,
 0.5,
 1,
 0.5,
 2.5,
 0.5,
 0.5,
 0.5,
 1,
 0.5,
 0.5,
 1,
 1.5,
 0.5,
 1,
 0.5,
 2,
 1,
 1.5,
 1,
 1,
 1,
 2.5,
 1,
 1,
 4.5,
 1,
 1.5,
 1,
 1,
 1,
 2,
 2.5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2.5,
 1,
 1,
 1.5,
 1,
 1,
 0.5,
 0.5,
 0.5,
 0.5]

In [72]:
#Checking length for add as a column in df_new Dataframe
len(list_new)

124

In [73]:
#Creat a new column "Weight slab charged by Courier Company (KG)"
df_new["Weight slab charged by Courier Company (KG)"]=list_new

In [74]:
df_new

,AWB Code,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,1.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,2.5
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,1.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5,1.0
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.607,1.0,0.5
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,0.515,1.0,0.5
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,0.689,1.0,0.5


In [75]:
#Recheck df2 dataframe for marging with df_new
df2

,Customer Pincode,Delivery Zone as per X
0,507101,d
1,486886,d
2,532484,d
3,143001,b
4,515591,d
...,...,...
119,325207,b
120,303702,b
121,313301,b
122,173212,e


In [76]:
#Checking Duplicate values
df2["Customer Pincode"].value_counts()

302017    5
313001    4
302031    2
302018    2
302002    2
         ..
284001    1
743263    1
410206    1
263139    1
342014    1
Name: Customer Pincode, Length: 108, dtype: int64

In [77]:
#Duplicate value is present

In [78]:
#Checking all duplicate values
df2[df2["Customer Pincode"].duplicated()]

,Customer Pincode,Delivery Zone as per X
16,140301,b
50,248001,b
72,302002,b
75,302017,b
77,313001,b
78,313001,b
82,313001,b
90,302017,b
101,335001,b
106,302031,b


In [79]:
df2[df2["Customer Pincode"]==302017]

,Customer Pincode,Delivery Zone as per X
74,302017,b
75,302017,b
90,302017,b
113,302017,b
117,302017,b


In [80]:
df2[df2["Customer Pincode"]==313001]

,Customer Pincode,Delivery Zone as per X
71,313001,b
77,313001,b
78,313001,b
82,313001,b


In [81]:
#we can see duplicate row is present

In [82]:
#drop duplicate data
df2.drop_duplicates(subset="Customer Pincode", inplace=True)

In [83]:
df2

,Customer Pincode,Delivery Zone as per X
0,507101,d
1,486886,d
2,532484,d
3,143001,b
4,515591,d
...,...,...
114,324008,b
115,302020,b
119,325207,b
120,303702,b


In [84]:
#we drop 16 duplicate columns

In [85]:
#marging df_new and df2 columns on "Customer Pincode"
df_new1=pd.merge(df_new,df2, on= "Customer Pincode")

In [86]:
#printing new marged file
df_new1

,AWB Code,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,1.5,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,1.0,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,2.5,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,1.0,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,0.5,d
...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091117904860,2001811039,0.68,121003,302020,d,Forward charges,90.2,0.488,0.5,1.0,b
120,1091121846136,2001811305,0.50,121003,302020,d,Forward charges,45.4,0.750,1.0,0.5,b
121,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5,1.0,b
122,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.607,1.0,0.5,b


In [87]:
#Checking null values
df_new1.isnull().sum()

AWB Code                                       0
Order ID                                       0
Total weight as per Courier Company (KG)       0
Warehouse Pincode                              0
Customer Pincode                               0
Delivery Zone charged by Courier Company       0
Type of Shipment                               0
Charges Billed by Courier Company (Rs.)        0
Total weight as per X (KG)                     0
Weight slab as per X (KG)                      0
Weight slab charged by Courier Company (KG)    0
Delivery Zone as per X                         0
dtype: int64

In [88]:
#No Null values present

In [89]:
df_new1.shape

(124, 12)

In [90]:
#Checking unique value in each columns
df_new1.nunique()

AWB Code                                       124
Order ID                                       124
Total weight as per Courier Company (KG)        54
Warehouse Pincode                                1
Customer Pincode                               108
Delivery Zone charged by Courier Company         3
Type of Shipment                                 2
Charges Billed by Courier Company (Rs.)         20
Total weight as per X (KG)                      68
Weight slab as per X (KG)                        7
Weight slab charged by Courier Company (KG)      7
Delivery Zone as per X                           3
dtype: int64

In [91]:
#Checking unique values in "Order ID" column
df_new1["Order ID"].value_counts()

2001821185    1
2001806229    1
2001806251    1
2001812650    1
2001807785    1
             ..
2001808542    1
2001808207    1
2001823564    1
2001806408    1
2001806776    1
Name: Order ID, Length: 124, dtype: int64

# Courier Company - Rates.csv

In [92]:
#Read Courier Company - Rates.csv file 
df5=pd.read_csv("Courier Company - Rates.csv")

In [93]:
df5

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [94]:
#Transpose the dataframe for beter visualization

In [95]:
df5.T

,0
fwd_a_fixed,29.5
fwd_a_additional,23.6
fwd_b_fixed,33.0
fwd_b_additional,28.3
fwd_c_fixed,40.1
fwd_c_additional,38.9
fwd_d_fixed,45.4
fwd_d_additional,44.8
fwd_e_fixed,56.6
fwd_e_additional,55.5


In [139]:
df_new1

,AWB Code,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,1.5,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,1.0,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,2.5,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,1.0,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,0.5,d
...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091117904860,2001811039,0.68,121003,302020,d,Forward charges,90.2,0.488,0.5,1.0,b
120,1091121846136,2001811305,0.50,121003,302020,d,Forward charges,45.4,0.750,1.0,0.5,b
121,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5,1.0,b
122,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.607,1.0,0.5,b


In [140]:
df_new1['Type of Shipment'].value_counts()

Forward charges            109
Forward and RTO charges     15
Name: Type of Shipment, dtype: int64

In [141]:
df_new1['Delivery Zone as per X'].value_counts()

b    79
d    38
e     7
Name: Delivery Zone as per X, dtype: int64

In [142]:
#Set "Order ID" as a index
df_new1=df_new1.set_index('Order ID')

In [143]:
df_new1.shape

(124, 11)

In [144]:
df_new1

,AWB Code,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X
Order ID,,,,,,,,,,,
2001806232,1091117222124,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,1.5,d
2001806273,1091117222194,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,1.0,d
2001806408,1091117222931,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,2.5,d
2001806458,1091117223244,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,1.0,b
2001807012,1091117229345,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,0.5,d
...,...,...,...,...,...,...,...,...,...,...,...
2001811039,1091117904860,0.68,121003,302020,d,Forward charges,90.2,0.488,0.5,1.0,b
2001811305,1091121846136,0.50,121003,302020,d,Forward charges,45.4,0.750,1.0,0.5,b
2001812941,1091118551656,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5,1.0,b


In [145]:
df_new1.index

Int64Index([2001806232, 2001806273, 2001806408, 2001806458, 2001807012,
            2001806686, 2001806885, 2001807058, 2001808118, 2001807186,
            ...
            2001809934, 2001810125, 2001810281, 2001811058, 2001810697,
            2001811039, 2001811305, 2001812941, 2001809383, 2001820978],
           dtype='int64', name='Order ID', length=124)

In [96]:
df5.T

,0
fwd_a_fixed,29.5
fwd_a_additional,23.6
fwd_b_fixed,33.0
fwd_b_additional,28.3
fwd_c_fixed,40.1
fwd_c_additional,38.9
fwd_d_fixed,45.4
fwd_d_additional,44.8
fwd_e_fixed,56.6
fwd_e_additional,55.5


In [97]:
#Determine "Expected Charge as per X (Rs.)" 

In [98]:
x_cost=[] #Creat a empty list
for i in df_new1.index:
    l=df_new1["Type of Shipment"][i]
    if l == 'Forward charges':     
        j=df_new1["Delivery Zone as per X"][i]
        if j== 'b':
            k=df_new1["Weight slab as per X (KG)"][i]
            if k<= 0.5:
                p=33   #Taken from df5( fwd_b_fixed	33.0    fwd_b_additional	28.3)
                x_cost.append(p)
            else:
                p=33+((k/0.5)-1)*28.5  #Taken from df5( fwd_b_fixed	33.0    fwd_b_additional	28.3)
                x_cost.append(p)
        elif j=='d':
            k=df_new1["Weight slab as per X (KG)"][i]
            if k<= 0.5:
                p=45.4    #fwd_d_fixed	45.4    fwd_d_additional	44.8
                x_cost.append(p)
            else:
                p=45.4+((k/0.5)-1)*44.8  #fwd_d_fixed	45.4    fwd_d_additional	44.8
                x_cost.append(p)                
        elif j=='e':
            k=df_new1["Weight slab as per X (KG)"][i]
            if k<= 0.5:
                p=k*56.6   #fwd_e_fixed	56.6    fwd_e_additional	55.5
                x_cost.append(p)
            else:
                p=56.6+((k/0.5)-1)*55.5  #fwd_e_fixed	56.6    fwd_e_additional	55.5
                x_cost.append(p)
                    
    else:
        
        j=df_new1["Delivery Zone as per X"][i]
        if j== 'b':
            k=df_new1["Weight slab as per X (KG)"][i]
            if k<= 0.5:
                p=(33+20.5) #df5( fwd_b_fixed	33.0    fwd_b_additional	28.3)  rto_b_fixed	20.5    rto_b_additional	28.3
                x_cost.append(p)
            else:
                p=(33+20.5)+(((k/0.5)-1)*(28.3+28.3))
                x_cost.append(p)
        elif j=='d':
            k=df_new1["Weight slab as per X (KG)"][i]
            if k<= 0.5:
                p=(45.4+41.3)  #fwd_d_fixed	45.4    fwd_d_additional	44.8  rto_d_fixed	41.3    rto_d_additional	44.8
                x_cost.append(p)
            else:
                p=(45.4+41.3)+(((k/0.5)-1)*(44.8+44.8))
                x_cost.append(p)                
        elif j=='e':
            k=df_new1["Weight slab as per X (KG)"][i]
            if k<= 0.5:
                p=(56.6+50.7)  #fwd_e_fixed	56.6    fwd_e_additional	55.5  rto_e_fixed	50.7    rto_e_additional	55.5
                x_cost.append(p)
            else:
                p=(56.6+50.7)+(((k/0.5)-1)*(55.5+55.5))
                x_cost.append(p) 
print("Code Succesfully Run")                

Code Succesfully Run


In [99]:
x_cost

[135.0,
 90.19999999999999,
 224.6,
 61.5,
 45.4,
 45.4,
 61.5,
 90.0,
 33,
 45.4,
 45.4,
 61.5,
 90.19999999999999,
 90.0,
 90.19999999999999,
 45.4,
 135.0,
 90.19999999999999,
 107.30000000000001,
 61.5,
 90.19999999999999,
 90.0,
 176.29999999999998,
 110.1,
 176.29999999999998,
 176.29999999999998,
 218.3,
 265.9,
 166.7,
 176.29999999999998,
 355.49999999999994,
 265.9,
 176.29999999999998,
 33,
 45.4,
 45.4,
 33,
 45.4,
 45.4,
 45.4,
 45.4,
 33,
 90.19999999999999,
 33,
 33,
 61.5,
 45.4,
 90.19999999999999,
 45.4,
 45.4,
 61.5,
 45.4,
 135.0,
 45.4,
 224.6,
 90.0,
 118.5,
 179.79999999999998,
 314.19999999999993,
 118.5,
 61.5,
 33,
 61.5,
 33,
 61.5,
 33,
 61.5,
 61.5,
 61.5,
 61.5,
 61.5,
 61.5,
 53.5,
 175.5,
 61.5,
 147.0,
 33,
 33,
 53.5,
 33,
 61.5,
 33,
 61.5,
 61.5,
 33,
 33,
 33,
 61.5,
 33,
 61.5,
 90.0,
 33,
 112.1,
 112.1,
 28.3,
 223.1,
 61.5,
 33,
 33,
 33,
 33,
 33,
 33,
 61.5,
 33,
 61.5,
 28.3,
 33,
 33,
 61.5,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 61.5,
 33

In [101]:
#Checking length
len(x_cost)

124

In [102]:
#Add a new column "Expected Charge as per X (Rs.)" by x_cost list
df_new1["Expected Charge as per X (Rs.)"]=x_cost

In [103]:
df_new1

,AWB Code,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,1.5,d,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,1.0,d,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,2.5,d,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,1.0,b,61.5
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,0.5,d,45.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091117904860,2001811039,0.68,121003,302020,d,Forward charges,90.2,0.488,0.5,1.0,b,33.0
120,1091121846136,2001811305,0.50,121003,302020,d,Forward charges,45.4,0.750,1.0,0.5,b,61.5
121,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5,1.0,b,33.0
122,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.607,1.0,0.5,b,110.1


In [158]:
df=df_new1

In [159]:
#Add a new column "Difference Between Expected Charges and Billed Charges (Rs.)"
df["Difference Between Expected Charges and Billed Charges (Rs.)"]=np.round(df["Expected Charge as per X (Rs.)"]-df["Charges Billed by Courier Company (Rs.)"],0)

In [160]:
df

,AWB Code,Order ID,Total weight as per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.),Total weight as per X (KG),Weight slab as per X (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.302,1.5,1.5,d,135.0,0.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,0.615,1.0,1.0,d,90.2,-0.0
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2.265,2.5,2.5,d,224.6,0.0
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,0.700,1.0,1.0,b,61.5,0.0
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.240,0.5,0.5,d,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091117904860,2001811039,0.68,121003,302020,d,Forward charges,90.2,0.488,0.5,1.0,b,33.0,-57.0
120,1091121846136,2001811305,0.50,121003,302020,d,Forward charges,45.4,0.750,1.0,0.5,b,61.5,16.0
121,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,0.500,0.5,1.0,b,33.0,-57.0
122,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.607,1.0,0.5,b,110.1,23.0


In [161]:
#Checking all columns name
df.columns

Index(['AWB Code', 'Order ID', 'Total weight as per Courier Company (KG)',
       'Warehouse Pincode', 'Customer Pincode',
       'Delivery Zone charged by Courier Company', 'Type of Shipment',
       'Charges Billed by Courier Company (Rs.)', 'Total weight as per X (KG)',
       'Weight slab as per X (KG)',
       'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X',
       'Expected Charge as per X (Rs.)',
       'Difference Between Expected Charges and Billed Charges (Rs.)'],
      dtype='object')

In [162]:
#Creating final data frame 
df_final=df.loc[:,['Order ID','AWB Code','Total weight as per X (KG)','Weight slab as per X (KG)','Total weight as per Courier Company (KG)','Weight slab charged by Courier Company (KG)','Delivery Zone as per X','Delivery Zone charged by Courier Company','Expected Charge as per X (Rs.)','Charges Billed by Courier Company (Rs.)','Difference Between Expected Charges and Billed Charges (Rs.)']]

# OUTPUT DATA - 1

In [163]:
df_final

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,-0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.5,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811039,1091117904860,0.488,0.5,0.68,1.0,b,d,33.0,90.2,-57.0
120,2001811305,1091121846136,0.750,1.0,0.50,0.5,b,d,61.5,45.4,16.0
121,2001812941,1091118551656,0.500,0.5,0.73,1.0,b,d,33.0,90.2,-57.0
122,2001809383,1091117614452,0.607,1.0,0.50,0.5,b,d,110.1,86.7,23.0


# OUTPUT DATA - 2

In [165]:
correctly_charged_order=df_final[df_final["Difference Between Expected Charges and Billed Charges (Rs.)"]==0]
correctly_charged_order

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,-0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.5,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
5,2001806686,1091117229555,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
6,2001806885,1091117229776,0.840,1.0,1.00,1.0,b,b,61.5,61.3,0.0
7,2001807058,1091117323112,1.168,1.5,1.15,1.5,b,b,90.0,89.6,0.0
8,2001808118,1091117435134,0.343,0.5,0.50,0.5,b,b,33.0,33.0,0.0
9,2001807186,1091117323812,0.500,0.5,0.50,0.5,d,d,45.4,45.4,0.0


In [166]:
f1=correctly_charged_order["Charges Billed by Courier Company (Rs.)"].sum(axis = 0, skipna = True)
print("Number of correctly charged order id :",correctly_charged_order.shape[0])
print("Total orders where X has been correctly charged :",f1)

Number of correctly charged order id : 22
Total orders where X has been correctly charged : 1826.9


# Total orders where X has been correctly charged : 1826.9

In [167]:
############################################################################################

In [168]:
overcharged_order=df_final[df_final["Difference Between Expected Charges and Billed Charges (Rs.)"]<0]
overcharged_order

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
33,2001806210,1091117221940,0.220,0.5,2.92,3.0,b,b,33.0,174.5,-142.0
34,2001806226,1091117222065,0.480,0.5,0.68,1.0,d,d,45.4,90.2,-45.0
35,2001806229,1091117222080,0.500,0.5,0.71,1.0,d,d,45.4,90.2,-45.0
36,2001806233,1091117222135,0.245,0.5,0.78,1.0,b,b,33.0,61.3,-28.0
37,2001806251,1091117222146,0.245,0.5,1.27,1.5,d,d,45.4,135.0,-90.0
...,...,...,...,...,...,...,...,...,...,...,...
116,2001810281,1091117805390,0.500,0.5,0.68,1.0,b,d,33.0,90.2,-57.0
117,2001811058,1091117905022,0.500,0.5,0.72,1.0,b,d,33.0,90.2,-57.0
118,2001810697,1091117807140,0.607,1.0,2.27,2.5,b,d,61.5,224.6,-163.0
119,2001811039,1091117904860,0.488,0.5,0.68,1.0,b,d,33.0,90.2,-57.0


In [169]:
f2=overcharged_order["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(axis = 0, skipna = True)
print("Number of overcharged order id :",overcharged_order.shape[0])
print("Total Orders where X has been overcharged :",f2)

Number of overcharged order id : 80
Total Orders where X has been overcharged : -4453.0


# Total Orders where X has been overcharged : 4453.0

In [170]:
##################################################################################

In [171]:
undercharged_order=df_final[df_final["Difference Between Expected Charges and Billed Charges (Rs.)"]>0]
undercharged_order

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
22,2001807976,1091117327496,0.721,1.0,0.70,1.0,d,d,176.3,172.8,3.0
23,2001812838,1091118547832,0.558,1.0,0.60,1.0,b,b,110.1,102.3,8.0
24,2001816684,1091119398844,0.920,1.0,0.99,1.0,d,d,176.3,172.8,3.0
25,2001817160,1091119630264,0.700,1.0,0.70,1.0,d,d,176.3,172.8,3.0
26,2001818390,1091120014461,0.841,1.0,0.80,1.0,e,e,218.3,213.5,5.0
27,2001821190,1091120959015,1.200,1.5,1.20,1.5,d,d,265.9,258.9,7.0
28,2001817093,1091121485824,1.357,1.5,1.30,1.5,b,b,166.7,151.1,16.0
29,2001823564,1091121666133,0.672,1.0,0.70,1.0,d,d,176.3,172.8,3.0
30,2001825261,1091121981575,1.557,2.0,1.60,2.0,d,d,355.5,345.0,10.0
31,2001811192,1091117957780,1.032,1.5,1.13,1.5,d,d,265.9,258.9,7.0


In [172]:
f3=undercharged_order["Difference Between Expected Charges and Billed Charges (Rs.)"].sum(axis = 0, skipna = True)
print("Number of undercharged order id :",undercharged_order.shape[0])
print("Total Orders where X has been undercharged",f3)

Number of undercharged order id : 22
Total Orders where X has been undercharged 551.0


# Total Orders where X has been undercharged 551.0

In [133]:
#####################################################################################

# Create A Dataframe For Output Data - 2

In [176]:
df_ex=pd.DataFrame({"Headings":["Total orders where X has been correctly charged","Total Orders where X has been overcharged","Total Orders where X has been undercharged"],"Count":[22,80,22],"Amount (Rs.)":[f1,f2,f3]})

In [177]:
df_ex

,Headings,Count,Amount (Rs.)
0,Total orders where X has been correctly charged,22,1826.9
1,Total Orders where X has been overcharged,80,-4453.0
2,Total Orders where X has been undercharged,22,551.0


# Save The Final Ouput In a Excel File

In [178]:
with pd.ExcelWriter('Cointab_Data_Analyst_Project_Final_Output_Data.xlsx') as writer:
    df_final.to_excel(writer, sheet_name='Cointab_Data_Analyst_Output_1')
    correctly_charged_order.to_excel(writer,sheet_name='correctly_charged_order_data')
    overcharged_order.to_excel(writer,sheet_name="overcharged_order_data")
    undercharged_order.to_excel(writer,sheet_name="undercharged_order_data")
    df_ex.to_excel(writer,sheet_name="final_output_2")

# Comparison with expected results

In [174]:
df_final[df_final["Order ID"]==2001806232]

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.3,1.5,d,d,135.0,135.0,0.0


In [175]:
df_final[df_final["Order ID"]==2001806210]

,Order ID,AWB Code,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
33,2001806210,1091117221940,0.22,0.5,2.92,3.0,b,b,33.0,174.5,-142.0


# N.B:- My result and Expected Result Both are same 

# All Output Data Link:
https://github.com/KBkoushik/cointab-data-analyst-project-solution.git

# %%%%%%%%%%%%:-   THANK YOU   -:%%%%%%%%%%%%%